# Client Example

In [1]:
from langserve import RemoteRunnable

In [2]:
from typing import Optional, List
from pydantic import BaseModel, Field

class Person(BaseModel):
    age: Optional[int] = Field(None, description="The age of the person in years.")
    name: Optional[str] = Field(None, description="The name of the person.")

In [3]:
runnable = RemoteRunnable("http://localhost:8000/extract_text/")

In [4]:
text = """
My name is Chester. I am 42 years old. My friend Jane is a year older than me.
"""

In [5]:
response = runnable.invoke({"text": text, "schema": Person.schema()})
response

{'data': [{'name': 'Chester', 'age': 42}]}

Add instructions:

In [6]:
instructions = "Redact all names using the characters `######`"

response = runnable.invoke(
    {
        "text": text,
        "schema": Person.schema(),
        "instructions": instructions,
    }
)
response

{'data': [{'name': 'Chester', 'age': 42}]}

Add few-shot examples:

In [7]:
instructions = "Redact all names using the characters `######`"
examples = [
    {
        "text": "My name is Grung. I am 100.",
        "output": [ {"age": 100, "name": "######", "hello": "meow"}] ,
    }
]

response = runnable.invoke(
    {
        "text": text,
        "schema": Person.schema(),
        "instructions": instructions,
        "examples": examples,
    }
)
response

{'data': [{'name': '######', 'age': 42}, {'name': 'Jane', 'age': 43}]}

## Persist extractors

In [8]:
import requests
from uuid import uuid4

In [9]:
url = "http://localhost:8000"
user_id = str(uuid4())  # indicates owner for extractor
headers = {"x-key": user_id}

In [10]:
data = {
    "name": "people_extractor",
    "description": "Extract references to people, having properties name and age.",
    "schema": Person.schema(),
    "instruction": "Redact all names using the characters `######`",
}

response = requests.post(f"{url}/extractors", json=data, headers=headers)
response

<Response [200]>

In [11]:
uuid = response.json()['uuid']
print(uuid)

fa60ccce-5637-41b4-ba1a-085a56d0fa5b


### Add examples

In [12]:
import json

examples = [
    {
        "text": "My name is Grung. I am 100.",
        "output": [Person(age=100, name="######").dict()],
    }
]

responses = []
for example in examples:
    create_request = {
        "extractor_id": uuid,
        "content": example["text"],
        "output": example['output'],
    }
    response = requests.post(f"{url}/examples", json=create_request, headers=headers)
    responses.append(response)

In [13]:
response = requests.get(f"{url}/examples?extractor_id={uuid}", headers=headers)
response

<Response [200]>

In [14]:
response.json()

[{'extractor_id': 'fa60ccce-5637-41b4-ba1a-085a56d0fa5b',
  'content': 'My name is Grung. I am 100.',
  'created_at': '2024-03-22T12:10:32.862261',
  'updated_at': '2024-03-22T12:10:32.862265',
  'output': [{'age': 100, 'name': '######'}],
  'uuid': '94c8a41a-7d33-4795-a6be-23e72fe6c4e4'}]

### Extract using persisted extractor

In [15]:
request_data = {"extractor_id": uuid, "text": text}
print(request_data)

{'extractor_id': 'fa60ccce-5637-41b4-ba1a-085a56d0fa5b', 'text': '\nMy name is Chester. I am 42 years old. My friend Jane is a year older than me.\n'}


In [16]:
response = requests.post(f"{url}/extract", data=request_data, headers=headers)
response

<Response [200]>

In [17]:
response.text

'{"data":[{"name":"######","age":42},{"name":"######","age":43}]}'